In [ ]:
# This notebook contains the code needed to interact with mySQL database
# ****Sections*****
# 1. Storing the Raw Data, from Quandl or from IB...
# 2. Storing Metadata (Costs data, Buy/Sell stats, Roll dates, Roll data, 
#         ...Historical Contango/Bacwardation statistics
# 3. Creating associated pysystemtrade Data object

In [10]:
# GENERATING and STORING RAW DATA
# Each Raw File will have the following format
# {Datetime, Open,High,Low,Settlement,Volume, Openinterest}
# Note: Quandl does not have consistent column naming across markets!

In [4]:
# Example... (Eurodollar)...
from dbutils.connect import *
import pandas as pd
from os import listdir
from os.path import isfile, join

hist_path = "quandl_history/"
maturity = 'EDOLLAR202003'
suffix = ".csv"
user = "root"
pwd = "admin"
database = "pkdemo"
host = "0.0.0.0"

dbmysql = opendb(username=user,password=pwd, database=database, port=3306, host=host)
dbmysql.connect()

# Load into database table the conents of filename
filename = hist_path + maturity + suffix
table = load_csv(dbmysql, filename,\
                 fields=['date','open','high','low','last','settle','volume'],\
                 fieldnames= ['Date','Open','High','Low','Last','Settle','Volume'],\
                 db_table='eurodollar')

/home/lope/envs/notebook/lib/python3.5/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'change' at row 1")
  result = self._query(query)
/home/lope/envs/notebook/lib/python3.5/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'last' at row 1")
  result = self._query(query)


In [7]:
'''from dbutils.connect import *
for row in EDOLLAR202003.select():
    print(row.date, row.high)'''

In [6]:
class Person(Model):
    name = CharField()
    birthday = DateField()
    is_relative = BooleanField()

    class Meta:
        database = dbmysql # This model uses the "people.db" database.

In [8]:
class Pet(Model):
    owner = ForeignKeyField(Person, related_name='pets')
    name = CharField()
    animal_type = CharField()

    class Meta:
        database = dbmysql # this model uses the "people.db" database

In [9]:
dbmysql.create_tables([Person, Pet])

In [10]:
from datetime import date
uncle_bob = Person(name='Bob', birthday=date(1960, 1, 15), is_relative=True)
uncle_bob.save() # bob is now stored in the database
grandma = Person.create(name='Grandma', birthday=date(1935, 3, 1), is_relative=True)
herb = Person.create(name='Herb', birthday=date(1950, 5, 5), is_relative=False)

In [11]:
for person in Person.select():
    print(person.name, person.is_relative)

Bob True
Grandma True
Herb False


In [18]:
from peewee import *
import playhouse
introspector = Introspector.from_database(dbmysql)
models = introspector.generate_models()
print(type(models))

NameError: name 'Introspector' is not defined

In [17]:
from peewee import *
from playhouse import *
help(playhouse)

Help on package playhouse:

NAME
    playhouse

PACKAGE CONTENTS
    apsw_ext
    berkeleydb
    csv_loader
    csv_utils
    dataset
    db_url
    djpeewee
    fields
    flask_utils
    gfk
    hybrid
    kv
    migrate
    pool
    postgres_ext
    read_slave
    reflection
    shortcuts
    signals
    sqlcipher_ext
    sqlite_ext
    sqlite_udf
    sqliteq
    test_utils

FILE
    /home/lope/envs/notebook/lib/python3.5/site-packages/playhouse/__init__.py




In [22]:
print("Pete")

Pete


In [89]:
class EDOLLAR202003(Model):
    date = DateField()
    open = FloatField()
    high = FloatField()
    low = FloatField()
    last = FloatField()
    settle = FloatField()
    volume = IntegerField()
    
    class Meta:
        database = dbmysql # This model uses the "people.db" database.
cur = dbmysql.create_table(EDOLLAR202003, safe=True)

In [34]:
for row in cur:
    print(row)

In [32]:
sq = EDOLLAR202003.select()
for row in sq:
    print(row.date)

In [36]:
print(cur)

In [38]:
for row in cur.fetchall():
    print(row)

In [39]:
print(cur)

In [58]:
cur2 = dbmysql.execute_sql('select * from EDOLLAR202003')
print("Pete")

Pete


In [49]:
import pandas as pd
df = pd.DataFrame.from_records((cur2, ['date', 'open']))
print(df.head(10))

      0     1
0  None  None
1  date  open


In [74]:
rows = pd.DataFrame(list('select * from person').dicts())
print(rows.tail(5))

AttributeError: 'list' object has no attribute 'dicts'

In [68]:
query = cur2.fetchall()
df = pd.DataFrame(list(query.dicts()))
print(df.tail(5))

AttributeError: 'tuple' object has no attribute 'dicts'

In [81]:
query = Person.select()
df = pd.DataFrame(list(query.dicts()))
df.tail()

,birthday,id,is_relative,name
0,1960-01-15,1,True,Bob
1,1935-03-01,2,True,Grandma
2,1950-05-05,3,False,Herb


In [85]:
myquery = EDOLLAR202003.select()
df = pd.DataFrame(list(myquery.dicts()))
df.tail()

""


In [119]:
help(load_csv)

Help on function load_csv in module playhouse.csv_utils:

load_csv(db_or_model, file_or_name, fields=None, field_names=None, has_header=True, sample_size=10, converter=None, db_table=None, pk_in_csv=False, **reader_kwargs)
    Load the contents of a CSV file into a database and return a model class
    suitable for working with the CSV data.
    
    :param db_or_model: a peewee Database instance or a Model class.
    :param file_or_name: the filename of the CSV file *or* a file handle.
    :param list fields: A list of peewee Field() instances appropriate to
        the values in the CSV file.
    :param list field_names: A list of names to use for the fields.
    :param bool has_header: Whether the first row of the CSV file is a header.
    :param int sample_size: Number of rows to introspect if fields are not
        defined.
    :param converter: A RowConverter instance to use.
    :param str db_table: Name of table to store data in (if not specified, the
        table name will be

In [83]:
print(type(query), type(myquery))

<class 'peewee.SelectQuery'> <class 'peewee.SelectQuery'>


In [101]:
hist_path = "quandl_history/"
maturity = 'EDOLLAR202003'
suffix = ".csv"
user = "root"
pwd = "admin"
database = "pkdemo"
host = "0.0.0.0"

dbmysql = opendb(username=user,password=pwd, database=database, port=3306, host=host)
dbmysql.connect()

# Load into database table the conents of filename
filename = hist_path + maturity + suffix
table = load_csv(dbmysql, filename,\
                 db_table='eurodollar')
my = load_csv(d)

InternalError: (1054, "Unknown column 'date' in 'field list'")

In [105]:
class edollar(Model):
    date = DateField()
    open = FloatField()
    high = FloatField()
    low = FloatField()
    last = FloatField()
    settle = FloatField()
    volume = FloatField()
    
    class Meta:
        database = dbmysql # This model uses pkdemo

In [106]:
table = load_csv(edollar, filename)

/home/lope/envs/notebook/lib/python3.5/site-packages/pymysql/cursors.py:166: Warning: (1364, "Field 'last' doesn't have a default value")
  result = self._query(query)
/home/lope/envs/notebook/lib/python3.5/site-packages/pymysql/cursors.py:166: Warning: (1364, "Field 'settle' doesn't have a default value")
  result = self._query(query)


/home/lope/envs/notebook/lib/python3.5/site-packages/pymysql/cursors.py:166: Warning: (1364, "Field 'open' doesn't have a default value")
  result = self._query(query)
/home/lope/envs/notebook/lib/python3.5/site-packages/pymysql/cursors.py:166: Warning: (1364, "Field 'low' doesn't have a default value")
  result = self._query(query)


In [121]:
class zedollar(Model):
    date = DateField()
    open = FloatField()
    high = FloatField()
    low = FloatField()
    last = FloatField()
    change = FloatField()
    settle = FloatField()
    volume = FloatField()
    openinterest = FloatField()
    
    class Meta:
        database = dbmysql # This model uses pkdemo
table = load_csv(zedollar, filename, field_names=['date','open','high','low','last','change','settle','volume','openinterest'])

/home/lope/envs/notebook/lib/python3.5/site-packages/pymysql/cursors.py:166: Warning: (1364, "Field 'last' doesn't have a default value")
  result = self._query(query)
/home/lope/envs/notebook/lib/python3.5/site-packages/pymysql/cursors.py:166: Warning: (1364, "Field 'change' doesn't have a default value")
  result = self._query(query)


/home/lope/envs/notebook/lib/python3.5/site-packages/pymysql/cursors.py:166: Warning: (1364, "Field 'open' doesn't have a default value")
  result = self._query(query)
/home/lope/envs/notebook/lib/python3.5/site-packages/pymysql/cursors.py:166: Warning: (1364, "Field 'low' doesn't have a default value")
  result = self._query(query)


In [122]:
query = zedollar.select()
df = pd.DataFrame(list(query.dicts()))
df.tail()

,change,date,high,id,last,low,open,openinterest,settle,volume
1741,0.005,2017-03-14,97.425,1742,97.400,97.375,97.375,274710.0,97.400,67693.0
1742,0.125,2017-03-15,97.550,1743,97.545,97.395,97.400,281523.0,97.525,150978.0
1743,0.005,2017-03-16,97.550,1744,97.505,97.490,97.545,288710.0,97.530,114919.0
1744,0.010,2017-03-17,97.560,1745,97.545,97.500,97.500,289607.0,97.540,45036.0
1745,0.040,2017-03-20,97.590,1746,97.590,97.540,97.550,290182.0,97.580,54458.0


In [124]:
df.rename(columns={"date":"DATETIME",\
                   "open":"OPEN",\
                   "high":"HIGH",\
                   "low":"LOW",\
                   "settle":"SETTLE",\
                   "volume":"VOLUME",\
    
                   "openinterest":"OPENINTEREST"},
          inplace=True)
df

,change,DATETIME,HIGH,id,last,LOW,OPEN,OPENINTEREST,SETTLE,VOLUME
0,0.000,2010-04-01,94.080,1,0.000,94.015,94.055,100.0,94.080,0.0
1,0.000,2010-04-05,93.970,2,0.000,93.840,93.970,150.0,93.940,0.0
2,0.000,2010-04-06,94.050,3,0.000,94.020,94.040,201.0,94.020,53.0
3,0.000,2010-04-07,94.130,4,0.000,94.030,0.000,251.0,94.130,50.0
4,0.000,2010-04-08,94.170,5,0.000,94.090,0.000,251.0,94.100,0.0
5,0.000,2010-04-09,94.100,6,0.000,94.050,94.050,254.0,94.100,3.0
6,0.000,2010-04-12,94.195,7,0.000,94.130,0.000,304.0,94.195,50.0
7,0.000,2010-04-13,94.275,8,0.000,94.215,94.245,304.0,94.265,0.0
8,0.000,2010-04-14,94.235,9,0.000,94.175,94.235,354.0,94.180,50.0
9,0.000,2010-04-15,94.190,10,0.000,94.130,94.150,404.0,94.165,50.0


In [125]:
df.head(6)

,change,DATETIME,HIGH,id,last,LOW,OPEN,OPENINTEREST,SETTLE,VOLUME
0,0.0,2010-04-01,94.08,1,0.0,94.015,94.055,100.0,94.08,0.0
1,0.0,2010-04-05,93.97,2,0.0,93.840,93.970,150.0,93.94,0.0
2,0.0,2010-04-06,94.05,3,0.0,94.020,94.040,201.0,94.02,53.0
3,0.0,2010-04-07,94.13,4,0.0,94.030,0.000,251.0,94.13,50.0
4,0.0,2010-04-08,94.17,5,0.0,94.090,0.000,251.0,94.10,0.0
5,0.0,2010-04-09,94.10,6,0.0,94.050,94.050,254.0,94.10,3.0


In [126]:
df.set_index('DATETIME',inplace=True)
df

,change,HIGH,id,last,LOW,OPEN,OPENINTEREST,SETTLE,VOLUME
DATETIME,,,,,,,,,
2010-04-01,0.000,94.080,1,0.000,94.015,94.055,100.0,94.080,0.0
2010-04-05,0.000,93.970,2,0.000,93.840,93.970,150.0,93.940,0.0
2010-04-06,0.000,94.050,3,0.000,94.020,94.040,201.0,94.020,53.0
2010-04-07,0.000,94.130,4,0.000,94.030,0.000,251.0,94.130,50.0
2010-04-08,0.000,94.170,5,0.000,94.090,0.000,251.0,94.100,0.0
2010-04-09,0.000,94.100,6,0.000,94.050,94.050,254.0,94.100,3.0
2010-04-12,0.000,94.195,7,0.000,94.130,0.000,304.0,94.195,50.0
2010-04-13,0.000,94.275,8,0.000,94.215,94.245,304.0,94.265,0.0
2010-04-14,0.000,94.235,9,0.000,94.175,94.235,354.0,94.180,50.0


In [128]:
df.index.values

array([datetime.date(2010, 4, 1), datetime.date(2010, 4, 5),
       datetime.date(2010, 4, 6), ..., datetime.date(2017, 3, 16),
       datetime.date(2017, 3, 17), datetime.date(2017, 3, 20)], dtype=object)

In [130]:
df.loc[['DATETIME']> datetime.date(2017,3,16)]

TypeError: unorderable types: list() > datetime.date()

In [142]:
df[:1].index[0]

datetime.date(2010, 4, 1)

In [143]:
dt = datetime.date(2010, 4, 1)

In [144]:
dt

datetime.date(2010, 4, 1)

In [146]:
df

,change,HIGH,id,last,LOW,OPEN,OPENINTEREST,SETTLE,VOLUME
DATETIME,,,,,,,,,
2010-04-01,0.000,94.080,1,0.000,94.015,94.055,100.0,94.080,0.0
2010-04-05,0.000,93.970,2,0.000,93.840,93.970,150.0,93.940,0.0
2010-04-06,0.000,94.050,3,0.000,94.020,94.040,201.0,94.020,53.0
2010-04-07,0.000,94.130,4,0.000,94.030,0.000,251.0,94.130,50.0
2010-04-08,0.000,94.170,5,0.000,94.090,0.000,251.0,94.100,0.0
2010-04-09,0.000,94.100,6,0.000,94.050,94.050,254.0,94.100,3.0
2010-04-12,0.000,94.195,7,0.000,94.130,0.000,304.0,94.195,50.0
2010-04-13,0.000,94.275,8,0.000,94.215,94.245,304.0,94.265,0.0
2010-04-14,0.000,94.235,9,0.000,94.175,94.235,354.0,94.180,50.0


In [151]:
df.loc[datetime.date(2017,2,28):datetime.date(2017,3,15)]

,change,HIGH,id,last,LOW,OPEN,OPENINTEREST,SETTLE,VOLUME
DATETIME,,,,,,,,,
2017-02-28,0.005,97.725,1732,97.630,97.605,97.690,265939.0,97.680,79446.0
2017-03-01,0.120,97.625,1733,97.565,97.540,97.625,261355.0,97.560,85357.0
2017-03-02,0.050,97.585,1734,97.520,97.490,97.565,262005.0,97.510,86976.0
2017-03-03,0.000,97.535,1735,97.520,97.455,97.515,269480.0,97.510,81760.0
2017-03-06,0.020,97.555,1736,97.525,97.510,97.515,270475.0,97.530,56943.0
2017-03-07,0.015,97.540,1737,97.500,97.500,97.525,271379.0,97.515,58056.0
2017-03-08,0.045,97.520,1738,97.450,97.435,97.505,293852.0,97.470,96110.0
2017-03-09,0.065,97.460,1739,97.390,97.390,97.445,303175.0,97.405,70047.0
2017-03-10,0.010,97.440,1740,97.420,97.375,97.390,309815.0,97.415,125755.0


In [153]:
d = {'col1': 'val1', 'col2': 'val2'}
sql = 'UPDATE table SET {}'.format(', '.join('{}=%s'.format(k) for k in d))
print(sql)

UPDATE table SET col2=%s, col1=%s


In [2]:
print(3)

3


In [34]:
import pandas as pd
roll_df = pd.read_csv("admin/roll_history.csv", dtype={'CARVER':str, 'PRICE_CONTRACT': str} )
markets = roll_df['CARVER']
mdf = pd.DataFrame(markets)
mdf.drop_duplicates(subset=['CARVER'],inplace=True)
print(mdf)
control_path = 'data_control/'
roll_df = pd.read_csv("admin/roll_history.csv", dtype={'CARVER':str, 'PRICE_CONTRACT': str} )
#print(roll_df.head(5))
roll_df.size
for market in mdf.itertuples():
    #print("---------------------")
    
    #print(market.CARVER)
    current_carry_file = control_path + market.CARVER + "_carrydata.csv"
    current_carry_df = pd.read_csv(current_carry_file,index_col=["DATETIME"], parse_dates=True,
                           dtype={"PRICE_CONTRACT":str, "CARRY_CONTRACT":str})
    #print("With nans: ", current_carry_df.size)
    
   # if market == '':
    #    print(current_carry_df.tail(5))

       CARVER
0         KR3
15        V2X
59    EDOLLAR
194       MXP
283      CORN
319   EUROSTX
334    GAS_US
465      PLAT
507       US2
546   LEANHOG
574       GBP
742       VIX
845       CAC
889    COPPER
979   CRUDE_W
1009     BOBL
1024    WHEAT
1042      JPY
1204   NASDAQ
1276     GOLD
1444      US5
1558  SOYBEAN
1576      AUD
1698    SP500
1777   PALLAD
1831     KR10
1846  LIVECOW
1864      NZD
1915    KOSPI
1930     US10
2070      SMI
2085      EUR
2159      OAT
2174      AEX
2218     BUND
2233      BTP
2248     US20


In [36]:
import pandas as pd
roll_df = pd.read_csv("admin/roll_history.csv", dtype={'CARVER':str, 'PRICE_CONTRACT': str} )
markets = roll_df['CARVER']
mdf = pd.DataFrame(markets)
mdf.drop_duplicates(subset=['CARVER'],inplace=True)
print(mdf)
control_path = 'data_control/'
roll_df = pd.read_csv("admin/roll_history.csv", dtype={'CARVER':str, 'PRICE_CONTRACT': str})
print(type(roll_df))

       CARVER
0         KR3
15        V2X
59    EDOLLAR
194       MXP
283      CORN
319   EUROSTX
334    GAS_US
465      PLAT
507       US2
546   LEANHOG
574       GBP
742       VIX
845       CAC
889    COPPER
979   CRUDE_W
1009     BOBL
1024    WHEAT
1042      JPY
1204   NASDAQ
1276     GOLD
1444      US5
1558  SOYBEAN
1576      AUD
1698    SP500
1777   PALLAD
1831     KR10
1846  LIVECOW
1864      NZD
1915    KOSPI
1930     US10
2070      SMI
2085      EUR
2159      OAT
2174      AEX
2218     BUND
2233      BTP
2248     US20
<class 'pandas.core.frame.DataFrame'>


In [2]:
current_carry_file = "/data/KR3_carrydata.csv"
current_carry_file = control_path + market.CARVER + "_carrydata.csv"
current_carry_df = pd.read_csv(current_carry_file,index_col=["DATETIME"], parse_dates=True,
                           dtype={"PRICE_CONTRACT":str, "CARRY_CONTRACT":str})
#print(current_carry_df.head(3))

NameError: name 'control_path' is not defined

In [32]:
if '1977-08-23' in current_carry_df.index:
    print("Yes")
else:
    print("No")

Yes


In [64]:
import pandas as pd
roll_df = pd.read_csv("admin/roll_history.csv", dtype={'CARVER':str, 'PRICE_CONTRACT': str} )
for row in roll_df.itertuples():
    symbol = row.CARVER
    date = row.DATETIME
    mat = row.PRICE_CONTRACT
    
    current_carry_file = "data_control/" + symbol + "_carrydata.csv"
    current_carry_df = pd.read_csv(current_carry_file,index_col=["DATETIME"], parse_dates=True,
                           dtype={"PRICE_CONTRACT":str, "CARRY_CONTRACT":str})
    download_file = "quandl_history/" + symbol + mat + ".csv" 
    print(download_file)
    download_df = pd.read_csv(download_file, usecols=["Date","Settle"])
    print(download_df.head(2))
    '''
    if date in download_df.index:
        
        print("Yes")
    else:
        print("No ", symbol, mat)
        
    #print(current_carry_df.head(1))
    
    download_file = "quandl_history/" + symbol + mat + ".csv" 
    print(download_file)
    download_df = pd.read_csv(download_file, index_col=["Date"], usecols=["Settle"])
    if not (date in download_file.index and date in current_carry_df.index):
        print(symbol + mat +" : NOTOK")
    else:
        "OK"
    '''

quandl_history/KR3201312.csv
         Date  Settle
0  2013-09-10  105.30
1  2013-09-12  105.41
quandl_history/KR3201403.csv
         Date  Settle
0  2013-12-16  105.58
1  2013-12-17  105.67
quandl_history/KR3201406.csv
         Date   Settle
0  2014-03-18  105.805
1  2014-03-19  105.860
quandl_history/KR3201409.csv
         Date  Settle
0  2014-06-17   106.2
1  2014-06-18   106.2
quandl_history/KR3201412.csv
         Date  Settle
0  2014-09-16  107.31
1  2014-09-17  107.28
quandl_history/KR3201503.csv
         Date   Settle
0  2014-12-16  108.055
1  2014-12-22  107.995
quandl_history/KR3201506.csv
         Date  Settle
0  2015-03-13  108.97
1  2015-03-16  109.00
quandl_history/KR3201509.csv
         Date  Settle
0  2015-06-16  109.14
1  2015-06-17  108.99
quandl_history/KR3201512.csv
         Date  Settle
0  2015-09-15  109.56
1  2015-09-16  109.50
quandl_history/KR3201603.csv
         Date  Settle
0  2015-12-14  109.35
1  2015-12-15  109.37
quandl_history/KR3201606.csv


         Date  Settle
0  2016-03-14  110.06
1  2016-03-15  110.05
quandl_history/KR3201609.csv
         Date  Settle
0  2016-09-14   110.6
1  2016-09-15   110.6
quandl_history/KR3201612.csv
         Date  Settle
0  2016-09-14  110.65
1  2016-09-15  110.65
quandl_history/KR3201703.csv
         Date  Settle
0  2016-09-21  110.66
1  2016-09-22  110.68
quandl_history/KR3201706.csv
         Date  Settle
0  2016-12-21  109.28
1  2016-12-22  109.39
quandl_history/V2X201311.csv
         Date  Settle
0  2013-09-09    21.8
1  2013-09-10    21.3
quandl_history/V2X201312.csv
         Date  Settle
0  2013-09-09   20.75
1  2013-09-10   20.35
quandl_history/V2X201401.csv


         Date  Settle
0  2013-09-09    22.7
1  2013-09-10    22.3
quandl_history/V2X201402.csv
         Date  Settle
0  2013-09-09   22.95
1  2013-09-10   22.60
quandl_history/V2X201403.csv
         Date  Settle
0  2013-09-09    23.3
1  2013-09-10    23.1
quandl_history/V2X201404.csv
         Date  Settle
0  2013-09-09   23.15
1  2013-09-10   22.85
quandl_history/V2X201405.csv
         Date  Settle
0  2013-09-19   22.55
1  2013-09-20   22.30
quandl_history/V2X201406.csv
         Date  Settle
0  2013-11-18   22.45
1  2013-11-19   22.50
quandl_history/V2X201407.csv
         Date  Settle
0  2013-12-02   22.05
1  2013-12-03   22.40
quandl_history/V2X201408.csv
         Date  Settle
0  2013-12-19   21.45
1  2013-12-20   21.60
quandl_history/V2X201409.csv
         Date  Settle
0  2014-02-06    22.3
1  2014-02-07    21.7
quandl_history/V2X201410.csv
         Date  Settle
0  2014-02-20   21.25
1  2014-02-21   21.10
quandl_history/V2X201411.csv
         Date  Settle
0  2014-04-10   21.20
1  201

quandl_history/V2X201506.csv
         Date  Settle
0  2014-10-22   20.35
1  2014-10-23   20.35
quandl_history/V2X201507.csv
         Date  Settle
0  2014-11-20    20.1
1  2014-11-21    20.0
quandl_history/V2X201508.csv
         Date  Settle
0  2014-12-18   21.35
1  2014-12-19   21.20
quandl_history/V2X201509.csv
         Date  Settle
0  2015-01-22   22.05
1  2015-01-23   22.05
quandl_history/V2X201510.csv
         Date  Settle
0  2015-02-19   22.50
1  2015-02-20   22.55
quandl_history/V2X201511.csv
         Date  Settle
0  2015-03-19    22.0
1  2015-03-20    21.8
quandl_history/V2X201512.csv
         Date  Settle
0  2015-04-16   20.85
1  2015-04-17   20.95
quandl_history/V2X201601.csv
         Date  Settle
0  2015-05-21   20.40
1  2015-05-22   20.55
quandl_history/V2X201602.csv
         Date  Settle
0  2015-06-18    21.4
1  2015-06-19    21.4
quandl_history/V2X201603.csv
         Date  Settle
0  2015-07-23    20.9
1  2015-07-24    20.9
quandl_history/V2X201604.csv
         Date  Settle

         Date  Settle
0  2016-07-21    24.0
1  2016-07-22    24.1
quandl_history/V2X201704.csv
         Date  Settle
0  2016-08-18   24.90
1  2016-08-19   24.85
quandl_history/V2X201705.csv
         Date  Settle
0  2016-09-22   24.65
1  2016-09-23   24.80
quandl_history/V2X201706.csv
         Date  Settle
0  2016-10-20    24.6
1  2016-10-21    24.7
quandl_history/EDOLLAR198412.csv
         Date  Settle
0  1983-09-23   88.92
1  1983-09-26   89.06
quandl_history/EDOLLAR198503.csv
         Date  Settle
0  1983-09-28   88.70
1  1983-09-29   88.68
quandl_history/EDOLLAR198506.csv
         Date  Settle
0  1984-01-10   88.32
1  1984-01-11   88.29
quandl_history/EDOLLAR198509.csv
         Date  Settle
0  1984-03-20   87.13
1  1984-03-21   87.17
quandl_history/EDOLLAR198512.csv
         Date  Settle
0  1984-06-19   85.61
1  1984-06-20   85.16


...

quandl_history/GBP201203.csv
         Date  Settle
0  2010-09-13  1.5343
1  2010-09-14  1.5499
quandl_history/GBP201206.csv
         Date  Settle
0  2010-12-13  1.5774
1  2010-12-14  1.5667
quandl_history/GBP201209.csv
         Date  Settle
0  2011-03-14  1.5997
1  2011-03-15  1.5925
quandl_history/GBP201212.csv
         Date  Settle
0  2011-06-13  1.6233
1  2011-06-14  1.6238
quandl_history/GBP201303.csv
         Date  Settle
0  2011-09-19  1.5613
1  2011-09-20  1.5645
quandl_history/GBP201306.csv
         Date  Settle
0  2011-12-19  1.5448
1  2011-12-20  1.5582
quandl_history/GBP201309.csv


         Date  Settle
0  2012-03-19  1.5830
1  2012-03-20  1.5808
quandl_history/GBP201312.csv
         Date  Settle
0  2012-06-18  1.5654
1  2012-06-19  1.5711
quandl_history/GBP201403.csv
         Date  Settle
0  2012-09-17  1.6183
1  2012-09-18  1.6203
quandl_history/GBP201406.csv
         Date  Settle
0  2012-12-17  1.6178
1  2012-12-18  1.6225
quandl_history/GBP201409.csv
         Date  Settle
0  2013-03-18  1.5089
1  2013-03-19  1.5089
quandl_history/GBP201412.csv
         Date  Settle
0  2013-06-17  1.5650
1  2013-06-18  1.5604
quandl_history/GBP201503.csv
         Date  Settle
0  2013-08-09  1.5478
1  2013-08-12  1.5430


quandl_history/GBP201506.csv
         Date  Settle
0  2013-08-12  1.5427
1  2013-08-13  1.5397
quandl_history/GBP201509.csv
         Date  Settle
0  2013-08-12  1.5430
1  2013-08-13  1.5398
quandl_history/GBP201512.csv
         Date  Settle
0  2013-08-12  1.5445
1  2013-08-13  1.5413
quandl_history/GBP201603.csv
         Date  Settle
0  2013-08-12  1.5460
1  2013-08-13  1.5428
quandl_history/GBP201606.csv
         Date  Settle
0  2013-08-12  1.5475
1  2013-08-13  1.5443
quandl_history/GBP201609.csv
         Date  Settle
0  2013-08-12  1.5492
1  2013-08-13  1.5458


quandl_history/GBP201612.csv
         Date  Settle
0  2013-08-12     0.0
1  2013-08-13     0.0
quandl_history/GBP201703.csv
         Date  Settle
0  2013-08-12  1.5521
1  2013-08-13  1.5487
quandl_history/GBP201706.csv
         Date  Settle
0  2013-08-12  1.5537
1  2013-08-13  1.5503
quandl_history/VIX200806.csv


ValueError: Usecols do not match names.

In [1]:
import pandas as pd
'''
roll_df = pd.read_csv("admin/roll_history.csv", dtype={'CARVER':str, 'PRICE_CONTRACT': str} )
for row in roll_df.itertuples():
    symbol = row.CARVER
    date = row.DATETIME
    mat = row.PRICE_CONTRACT
    
    if symbol == 'VIX':
    
        download_file = "quandl_history/" + symbol + mat + ".csv" 
        print(download_file)
        download_df = pd.read_csv(download_file)
        download_df.rename(columns={'Trade Date':'Date'},inplace=True)
        download_df.to_csv(download_file)
        print(download_df.head(2))
'''

'\nroll_df = pd.read_csv("admin/roll_history.csv", dtype={\'CARVER\':str, \'PRICE_CONTRACT\': str} )\nfor row in roll_df.itertuples():\n    symbol = row.CARVER\n    date = row.DATETIME\n    mat = row.PRICE_CONTRACT\n    \n    if symbol == \'VIX\':\n    \n        download_file = "quandl_history/" + symbol + mat + ".csv" \n        print(download_file)\n        download_df = pd.read_csv(download_file)\n        download_df.rename(columns={\'Trade Date\':\'Date\'},inplace=True)\n        download_df.to_csv(download_file)\n        print(download_df.head(2))\n'

In [5]:
import pandas as pd
roll_df = pd.read_csv("admin/roll_history.csv", dtype={'CARVER':str, 'PRICE_CONTRACT': str} )
list = []
for row in roll_df.itertuples():
    symbol = row.CARVER
    date = row.DATETIME # Check if this is in the downloaded history data.
    mat = row.PRICE_CONTRACT
    download_file = "quandl_history/" + symbol + mat + ".csv" 
    download_df = pd.read_csv(download_file, index_col=['Date'])
    if not(date in download_df.index):
        list.append([date, symbol, mat])
        #print(symbol, mat)
        #print("------------------")
        
print(list)

[['2016-06-20', 'KR3', '201609'], ['2013-03-26', 'V2X', '201311'], ['2016-06-20', 'KR10', '201609'], ['2016-09-13', 'KR10', '201612'], ['2016-09-05', 'KOSPI', '201612'], ['2013-03-22', 'SMI', '201312'], ['2013-07-26', 'AEX', '201310'], ['2014-12-19', 'AEX', '201501'], ['2015-01-16', 'AEX', '201502']]


In [14]:
import datetime
import pandas as pd
list = [['2016-06-20', 'KR3', '201609'], \
        ['2013-03-26', 'V2X', '201311'], \
        ['2016-06-20', 'KR10', '201609'],\
        ['2016-09-13', 'KR10', '201612'],\
        ['2016-09-05', 'KOSPI', '201612'],\
        ['2013-03-22', 'SMI', '201312'],\
        ['2013-07-26', 'AEX', '201310'],\
        ['2014-12-19', 'AEX', '201501'],\
        ['2015-01-16', 'AEX', '201502']]

for array in list:
    date = datetime.datetime.strptime(array[0], "%Y-%m-%d")
    start_date = date - datetime.timedelta(days=1000)
    print(start_date)
    print(end_date)
    end_date = date + datetime.timedelta(days=1000)
    
    download_file = "quandl_history/" + array[1] + array[2] + ".csv" 
    download_df = pd.read_csv(download_file, index_col=['Date'])
    
    print("-----------", array[0], array[1], array[2], "-------------")
    print(download_df.loc[:])

2013-09-24 00:00:00
2019-03-17 00:00:00
----------- 2016-06-20 KR3 201609 -------------
            Settle  Change    Open    High     Low  Trading Volume  \
Date                                                                 
2016-09-14  110.60   30.00  110.63  110.67  110.57        112142.0   
2016-09-15  110.60   30.00  110.63  110.67  110.57        112142.0   
2016-09-16  110.60   30.00  110.63  110.67  110.57        112142.0   
2016-09-19  110.58   -0.02  110.59  110.65  110.58         49606.0   
2016-09-20  110.59    0.01  110.60  110.60  110.57         22175.0   

            Open Interest  
Date                       
2016-09-14       198459.0  
2016-09-15       198459.0  
2016-09-16       198459.0  
2016-09-19        94217.0  
2016-09-20            0.0  
2010-06-30 00:00:00
2019-03-17 00:00:00
----------- 2013-03-26 V2X 201311 -------------
             Open   High    Low  Settle   Volume  Open Interest
Date                                                           
2013-09-0

In [15]:
import pandas as pd
destination_path = 'data_control/'
roll_df = pd.read_csv("admin/roll_history.csv", dtype={'CARVER':str, 'PRICE_CONTRACT': str} )
list = []
for row in roll_df.itertuples():
    symbol = row.CARVER
    date = row.DATETIME # Check if this is in the downloaded history data.
    mat = row.PRICE_CONTRACT
    current_carry_file = destination_path + symbol + "_carrydata.csv"

    current_price_df = pd.read_csv(current_carry_file,index_col=["DATETIME"], parse_dates=True,
                           dtype={"PRICE":str, "PRICE_CONTRACT":str})
    
    
    filtered_current_df = current_price_df[current_price_df['PRICE_CONTRACT'] == mat]
    flsize = filtered_current_df.size
    first = filtered_current_df[:1].index[0]
    last = filtered_current_df[-1:].index[0]
    download_file = "quandl_history/" + symbol + mat + ".csv" 
    download_df = pd.read_csv(download_file, index_col=['Date'])
    dlsize = download_df[first:last].size
    print(symbol, mat, "carry: ", flsize, "download: ", dlsize)

TypeError: Cannot compare type 'Timestamp' with type 'str'

In [1]:
from dbutils.connect import *
import pandas as pd
from os import walk
from os import listdir
from os.path import isfile, join


hist_path = "/home/lope/quandl_history/"
hist_files = [f for f in listdir("/home/lope/quandl_history/") 
                 if isfile(join("/home/lope/quandl_history/", f))]
hist_files

['WHEAT201712.csv']

In [2]:
listdir("/home/lope/quandl_history/")

['WHEAT201712.csv']

In [1]:
(4, 'Pete')

(4, 'Pete')

In [8]:
type([])

str